* 抓包利器Fiddler<br/>
* Cookies与保持登录<br/>
* Cookie模拟登录<br/>
* 模拟登录新浪微博

###1. Fiddler介绍
电脑与互联网之间的通信是通过数据包交换来实现的。<br/>
Fiddler可以从中间对数据进行拦截<br/>
拷贝一份数据再将数据发送给目的端<br/>
同类软件还有WireShark

In [38]:
from IPython.display import IFrame
IFrame('Fiddler.png', width = 700, height = 300)

安装Fiddler<br/>
一直点击“下一步”，配置：winConfig——勾选IE——浏览网页——拖过拖拽看数据

###2. Cookie
Cookie（复数Cookies），中文名称为小型文本文件。<br/>
某些网站为了辨别用户身份而存储在用户本地终端（client side）上的数据（通常加密过）。<br/>
当登录一个网站，往往需要输入用户名和密码，可以选择“下次自动登录”<br/>
如果选择，下一次访问时，用户可以跳过输入用户名、密码阶段实现直接登录<br/>
因为上一次登录时，服务器发送登录凭据的Cookie到用户端，第二次登录时，如果Cookie尚未到期，浏览器会发送该Cookie。

* Fiddler获取Cookies（服务器接收用户提交数据后返回的小型文本文件）
* requests提交Cookies<br/>
cookie = {'Cookie': 'XXXXXXXXXXX'}<br/>
html = requests.get(url, cookies = cookie)

###3. 模拟登录新浪微博：分析POST向服务器传输的数据
打开Firefox，输入weibo.cn<br/>
右键“查看元素”——“网络”——查看方法为POST的参数情况<br/>
**表单数据**
* mobile（邮箱）<br/>
* password_xxxx（密码，后面的数字是随机的，需要寻找）<br/>
* code（验证码）<br/>
* remember（下次自动登录）<br/>
* backURL（返回网址）<br/>
* backTitle（"微博"）<br/>
* tryCount（尝试次数，一次登录成功即为0）<br/>
* vk（一段字符串，需要寻找）<br/>
* capId（一段字符串，需要寻找）<br/>
* submit（"登录"）<br/>

观察POST中“消息头”——“请求网址”，分析其结构<br/>
http://login.weibo.cn/login/?rand=942880244&backURL=http://weibo.cn/kaikai0818?page=1&backTitle=微博&vt=4&revalid=2&ns=1<br/>
网址中的元素（rand, backURL, backTitle, vt, revalid, ns）集中在“网络”——“参数”——“查询字符串”中<br/>

* **核心代码：**<br/>
* data = {'mobile': 'XXXXX', password: 'XXXXXX', ……, 'submit': u'登录'}
* html = requests.post(url, data = data)

In [16]:
import requests, re
from lxml import etree

url = 'http://weibo.cn/kaikai0818?page=1'
# url为你需要抓取的微博首页地址，本处以演员王凯的微博首页为例
url_login = 'http://login.weibo.cn/login/'
# url_login为请求网址的开始部分

html = requests.get(url_login).content
selector = etree.HTML(html)
password = selector.xpath('//input[@type="password"]/@name')[0]
code = re.search('img src="(.*?)" alt=', html, re.S).group(1)
vk = selector.xpath('//input[@name="vk"]/@value')[0]
capId = selector.xpath('//input[@name="capId"]/@value')[0]
action = re.search('<br/><form action="(.*?)" method="post"><div>', html, re.S).group(1).replace('amp;', '')
print password
print code
print vk
print capId
print action

new_url = url_login + action
print new_url
data = {
    'mobile': 'luochxxxxxxxxxxx@foxmail.com', 
    password: 'lxxxxxxxxx6', 
    'code': '', 
    # 需要图像识别技术
    'remember': 'on', 
    'backURL': 'http://weibo.cn/kaikai0818?page=1', 
    'backTitle': u'微博', 
    'tryCount': '', 
    'vk': vk, 
    'capId': capId, 
    'submit': u'登录'
}

new_html = requests.post(new_url, data = data).content
print new_html + '\n\n'
new_selector = etree.HTML(new_html)
content = new_selector.xpath('//body/div')
for each in content:
    text = each.xpath('string(.)')
    print text

password_7002
http://weibo.cn/interface/f/ttt/captcha/show.php?cpt=4_50fb08d84d730490
7002_25f6_2160722277
4_50fb08d84d730490
?rand=1875698391&backURL=http%3A%2F%2Fweibo.cn&backTitle=%E6%89%8B%E6%9C%BA%E6%96%B0%E6%B5%AA%E7%BD%91&vt=1
http://login.weibo.cn/login/?rand=1875698391&backURL=http%3A%2F%2Fweibo.cn&backTitle=%E6%89%8B%E6%9C%BA%E6%96%B0%E6%B5%AA%E7%BD%91&vt=1
<?xml version="1.0" encoding="UTF-8"?><!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd"><html xmlns="http://www.w3.org/1999/xhtml"><head><meta http-equiv="Content-Type" content="text/html; charset=utf-8" /><meta http-equiv="Cache-Control" content="no-cache"/><meta id="viewport" name="viewport" content="width=device-width,initial-scale=1.0,minimum-scale=1.0, maximum-scale=2.0" /><link rel="icon" sizes="any" mask href="http://h5.sinaimg.cn/upload/2015/05/15/28/WeiboLogoCh.svg" color="black"><meta name="MobileOptimized" content="240"/><title>微博</title><style type="tex

###4. 当验证码遇阻，需要使用Fiddler

####1）单页分解
网页保存在本地

In [37]:
from lxml import etree
f = open('C://Users//Luo Chen//Desktop//4_1.html', 'r').read()
html = etree.HTML(f)
# print html
field = html.xpath('//div[@class="c"]')
num = 1
for each in field:
    content = each.xpath('div[1]')
    pic_field = each.xpath('div[2]')
    retweet_field = each.xpath('div[3]')
    
    if len(content) != 0:
        for each_1 in content:
            print 'Text, etc ' + str(num) + ': ' + each_1.xpath('string(.)')
    else:
        continue
        
    if len(pic_field) != 0:
        for each_2 in pic_field:
            pic_url = each_2.xpath('a/img/@src')[0]
            others = each_2.xpath('string(.)')
            print 'Pic, etc ' + str(num) + ': ' + pic_url + others
    else:
        print ''
        
    if len(retweet_field) != 0:
        for each_3 in retweet_field:
            print 'Retweet, etc ' + str(num) + ': ' + each_3.xpath('string(.)') + '\n'
    else:
        print ''
        
    num += 1

Text, etc 1: #电视剧欢乐颂# 4.18  等你！ [组图共2张]
Pic, etc 1: http://ww3.sinaimg.cn/wap180/4c2e175bgw1f2ogppwhu5j215o1k67wh.jpg 原图 赞[137863] 转发[59784] 评论[37355] 收藏 04月07日 21:33

Text, etc 2: 生日快乐@陈乔恩
Pic, etc 2: http://ww2.sinaimg.cn/wap180/4c2e175bjw1f2kqn03smsj21w02iou12.jpg 原图 赞[269112] 转发[27024] 评论[21950] 收藏 04月04日 16:14

Text, etc 3: #放弃我抓紧我# 钟楼在身后，怪人在哪里？ [组图共2张]
Pic, etc 3: http://ww1.sinaimg.cn/wap180/4c2e175bjw1f2ij0g51w0j22io1w0hdu.jpg 原图 赞[169225] 转发[48594] 评论[37158] 收藏 04月02日 18:30

Text, etc 4: 阴雨天也影响不了我们撒欢的心[哈哈]#放弃我抓紧我#
Pic, etc 4: http://ww2.sinaimg.cn/wap180/4c2e175bjw1f2g8mpgibwj21ho1zkb29.jpg 原图 赞[233490] 转发[48280] 评论[41228] 收藏 03月31日 18:50

Text, etc 5: 󾓧 [组图共2张]
Pic, etc 5: http://ww4.sinaimg.cn/wap180/4c2e175bjw1f2dtayqj5mj22ek38onp6.jpg 原图 赞[125793] 转发[72886] 评论[39655] 收藏 03月29日 16:29 来自360手机奇酷系列

Text, etc 6: 转发了 张晓谦ZHaNG 的微博:我们家的妩媚昨天晚上在北京市朝阳区十里堡石佛营附近走丢了，她跟了我四年，实在舍不得，她背后有一小块疤，性格调皮，毛发较少，大长腿，如果有好心的朋友发现了她，一定拨打18601240037，必有重谢！@霹雳爷们儿 @张晓谦粉丝团 [组图共4张]
Pic, etc 6: http://ww2.sinai

####2）多页操作
利用Fiddler先获取Cookie<br/>
* 由于Fiddler已经配置了IE，但是IE上输入验证码不可能成功，可以先登录weibo.com，再连接weibo.cn获取Cookie

再续接“单页分解”所习知识

####bytes与bytearray
* Bytearray<br/>
bytearray类型是一个**可变的**序列，bytearray(source, encoding, errors)，返回一个byte数组<br/>
参数source<br/>
如果source为整数，则返回一个长度为source的初始化数组<br/>
如果source为字符串，则按照指定的encoding将字符串转换为字节序列<br/>
如果source为可迭代类型，则元素必须为[0 ,255]中的整数<br/>
如果source为与buffer接口一致的对象，则此对象也可以被用于初始化bytearray<br/>
<br/>
* Bytes<br/>
返回一个新的数组对象，这个数组对象**不能**对数组元素进行修改<br/>
每个元素值范围: 0 <= x < 256<br/>
bytes函数与bytearray函数主要区别是**bytes函数产生的对象的元素不能修改，而bytearray函数产生的对象的元素可以修改**<br/>
因此，除了可修改的对象函数跟bytearray函数不一样之外，其它使用方法全部是相同的。最后它的参数定义方式也与bytearray函数是一样的。

In [1]:
import requests
from lxml import etree

cookie = { 'Cookie': '_T_WM=7f304475799e76fd5257d09b69902e44; SUHB=0BLpnOTl6bKyyd; SUB=_2A256CYMjDeRxGeRL61cU-CnNwzmIHXVZ9S1rrDV6P\
UJbstANLWzhkW1LHet8L7ThWmkn1bh7AopSe8sqlud1qw..; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9WFBiFz3du9KKOU.EYQjlJz.5JpX5o2p; SSOLogi\
nState=1460532083'}
i = int(raw_input('How many pages do you want?(INTEGER ALLOWED)  '))
url = 'http://weibo.cn/bowu?page='
num = 1
for i in range(640, i + 1):
    new_url = url + str(i)
    html_1 = requests.get(new_url, cookies = cookie)
    html = bytes(bytearray(html_1.text, encoding = 'utf-8'))
    selector = etree.HTML(html)
    field = selector.xpath('//div[@class="c"]')
    for each in field:
        content = each.xpath('div[1]')
        pic_field = each.xpath('div[2]')
        retweet_field = each.xpath('div[3]')

        if len(content) != 0:
            for each_1 in content:
                print 'Text, etc ' + str(num) + ': ' + each_1.xpath('string(.)')
        else:
            continue

        if len(pic_field) != 0:
            for each_2 in pic_field:
                pic_url = each_2.xpath('a/img/@src')
                if len(pic_url) != 0:
                    pic_url_1 = each_2.xpath('a/img/@src')[0]
                    others = each_2.xpath('string(.)')
                    print 'Pic, etc ' + str(num) + ': ' + pic_url_1 + others
                else:
                    others = each_2.xpath('string(.)')
                    print 'Pic, etc ' + str(num) + ': ' + others 
        else:
            print ''

        if len(retweet_field) != 0:
            for each_3 in retweet_field:
                print 'Retweet, etc ' + str(num) + ': ' + each_3.xpath('string(.)') + '\n'
        else:
            print ''
        num += 1

How many pages do you want?(INTEGER ALLOWED)  712
Text, etc 1: 转发了 昆虫学liuye 的微博:甲虫里的活宝石 有一些步甲（甲虫的一类）从体形上看算是整个步甲家族里的巨人，而且浑身披着色彩鲜艳、闪着金属光泽的甲胄，活像一枚枚亮丽的宝石。由于这类步甲个体很大，俗称大步甲。
Pic, etc 1: http://ww2.sinaimg.cn/wap180/9373d2b4jw1dn5n23f2izj.jpg 原图 赞[432] 原文转发[112] 原文评论[1010]
Retweet, etc 1: 转发理由:中国是大步甲资源最多的国家之一  赞[0] 转发[6] 评论[9] 收藏 2011-12-03 10:53:15 来自微博 weibo.com

Text, etc 2: 转发了 细细细细细龙 的微博:博物给力啊。有我爱的水晶下虾。还有整人玩的苍耳。还有最最最最最最最爱的GaGa.[给力]@博物杂志
Pic, etc 2: http://ww1.sinaimg.cn/wap180/69a1b379jw1dnoiqttld3j.jpg 原图 赞[0] 原文转发[3] 原文评论[2]
Retweet, etc 2: 转发理由:啊哈，够贴近大家的生活吧！  赞[1] 转发[0] 评论[4] 收藏 2011-12-03 10:43:15 来自微博 weibo.com

Text, etc 3: 转发了 全球奇闻趣事 的微博:【美丽而最致命的蛇】洛杉矶的摄影师 Mark Laita 周游美国和中美洲，拍摄了一些世界上最致命的蛇，这系列作品也被命名为“蛇形”。他说：蛇的吸引力，与它们不可以预知的神秘性质是共存的，虽然它们的美丽会令危险提高，但仍旧如此着迷。最后一图在哥斯达黎加拍摄，非洲最长毒蛇，4.2米。 
Pic, etc 3: http://ww1.sinaimg.cn/wap180/604e48d0jw1dnn8a5iyr5j.jpg 原图 赞[0] 原文转发[814] 原文评论[173]
Retweet, etc 3: 转发理由:亮点在最后一句。他是和马达加斯加搞混了吗？//@Botia: 非洲最长毒蛇居然在哥斯达黎加，这么好笑  赞[0] 转发[3] 评论[5] 收藏 2011-12-02 14:06: